In [1]:
import numpy as np
from numpy.fft import fft
from scipy.stats import skew, kurtosis 
import pandas as pd
import glob
import matplotlib.pyplot as plt
import os
import struct
import glob
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, classification_report, accuracy_score


import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import struct
from numpy.fft import fft
from scipy.stats import skew, kurtosis
import torch
from torch import nn
from torch.nn import functional as F
from torch import optim

# Check if CUDA is available and set the device to GPU if it is, otherwise CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

class ComplexNumbersDataset(Dataset):
    def __init__(self, filepath, sequence_length, max_samples=None, for_training=True, process_method='data1'):
        self.filepath = filepath
        self.sequence_length = sequence_length
        self.max_samples = max_samples
        self.for_training = for_training
        self.process_method = process_method
        self.samples = self.load_samples()
        self.samples_per_sequence = self.sequence_length
        #self.samples_per_sequence = self.sequence_length if for_training else 1

    def load_samples(self):
        samples = []
        with open(self.filepath, 'rb') as binary_file:
            while True:
                if self.max_samples and len(samples) >= self.max_samples:
                    break
                binary_data = binary_file.read(8)
                if not binary_data:
                    break
                decoded_data = struct.unpack('ff', binary_data)
                if decoded_data[0] == 0 and decoded_data[1] == 0:
                    continue
                samples.append(f"{decoded_data[0]}+{decoded_data[1]}j")
        return samples

    def process_data1(self, sequence_samples):
        #print('sequence_samples:', sequence_samples)
        #print('len(sequence_samples):', len(sequence_samples))
        real_parts = []
        imag_parts = []
        for sample in sequence_samples:
            # Remove potential unwanted characters (spaces, etc.)
            sample = sample.replace(" ", "").replace("+-", "-")
            try:
                c = complex(sample)
                real_parts.append(c.real)
                imag_parts.append(c.imag)
            except ValueError:
                print(f"Failed to convert: {sample}")  # This will show which string failed
                raise
        real_parts = np.array(real_parts)
        imag_parts = np.array(imag_parts)
        # Normalize
        real_parts = (real_parts - np.mean(real_parts)) / np.std(real_parts)
        imag_parts = (imag_parts - np.mean(imag_parts)) / np.std(imag_parts)

        # Combining real and imaginary parts
        X = np.stack((real_parts, imag_parts), axis=1)  # Shape: (sequence_length, 2)
        return torch.tensor(X, dtype=torch.float32)

    def process_data2(self, sequence_samples):
        samples_array = np.array([complex(sample) for sample in sequence_samples], dtype=np.complex64)
        samples_fft = fft(samples_array)
        real_parts = np.real(samples_fft)
        imag_parts = np.imag(samples_fft)
        
        # Normalization
        epsilon = 1e-10
        real_parts_normalized = (real_parts - np.mean(real_parts)) / (np.std(real_parts) + epsilon)
        imag_parts_normalized = (imag_parts - np.mean(imag_parts)) / (np.std(imag_parts) + epsilon)
        
        # Feature extraction
        features = np.column_stack((
            np.mean(real_parts_normalized), np.std(real_parts_normalized), skew(real_parts_normalized),
            kurtosis(real_parts_normalized), np.mean(imag_parts_normalized), np.std(imag_parts_normalized),
            skew(imag_parts_normalized), kurtosis(imag_parts_normalized)
        ))
        
        return torch.tensor(features, dtype=torch.float32).reshape(self.samples_per_sequence, -1)

    def __len__(self):
        return len(self.samples) // self.samples_per_sequence

    def __getitem__(self, idx):
        start_idx = idx * self.samples_per_sequence
        end_idx = start_idx + self.samples_per_sequence
        sequence_samples = self.samples[start_idx:end_idx]
        
        if self.process_method == 'data1':
            X = self.process_data1(sequence_samples)
        elif self.process_method == 'data2':
            X = self.process_data2(sequence_samples)
        else:
            raise ValueError("Invalid process method specified.")
        if self.for_training:
            return X, X
        else:
            return X
#------------------------------------------------------------------------------------------------------
class SelfAttentionLayer(nn.Module):
    def __init__(self, embed_size, num_heads):
        super(SelfAttentionLayer, self).__init__()
        self.multihead_attention = nn.MultiheadAttention(embed_dim=embed_size, num_heads=num_heads)

    def forward(self, inputs):
        # Expecting inputs of shape (sequence_length, batch_size, embed_size)
        attn_output, _ = self.multihead_attention(inputs, inputs, inputs)
        # Output shape is the same as input shape (sequence_length, batch_size, embed_size)
        return attn_output

class LSTMAutoencoder(nn.Module):
    def __init__(self, sequence_length, input_dim, lstm_dim=50, num_heads=2):
        super(LSTMAutoencoder, self).__init__()
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.lstm_dim = lstm_dim
        
        # Encoder
        self.encoder_lstm1 = nn.LSTM(input_dim, lstm_dim, batch_first=True)
        self.self_attention = SelfAttentionLayer(embed_size=lstm_dim, num_heads=num_heads)
        self.encoder_lstm2 = nn.LSTM(lstm_dim, lstm_dim, batch_first=True)
        
        # Decoder
        self.decoder_lstm1 = nn.LSTM(lstm_dim, lstm_dim, batch_first=True)
        self.decoder_lstm2 = nn.LSTM(lstm_dim, lstm_dim, batch_first=True)
        self.decoder_linear = nn.Linear(lstm_dim, input_dim)
        
    def forward(self, x):
        # Encoder
        x, (hidden_state, cell_state) = self.encoder_lstm1(x)
        # Permute x to match MultiheadAttention input shape expectations
        x = x.permute(1, 0, 2)  # Shape: (sequence_length, batch_size, embed_size)
        x = self.self_attention(x)
        # Permute x back to match LSTM input shape expectations
        x = x.permute(1, 0, 2)  # Shape: (batch_size, sequence_length, embed_size)
        x, (hidden_state, cell_state) = self.encoder_lstm2(x)
        
        # Decoder
        x, (hidden_state, cell_state) = self.decoder_lstm1(x)
        x, (hidden_state, cell_state) = self.decoder_lstm2(x)
        # Apply Linear layer to each time step
        x = self.decoder_linear(x)
        
        return x

# Example of initializing the model
sequence_length = 10

feature_dim = 2
lstm_dim = 50
num_heads = 2
epsilon_std = 0.1
model = LSTMAutoencoder(sequence_length=10, input_dim=feature_dim).to(device)
print(model)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
loss_function = nn.MSELoss()


# Model Training
batch_size = 20
max_train_samples = 25000
train_steps = max_train_samples // (batch_size * sequence_length)
max_samples = 25000  # Maximum samples to read (or None to read all)
max_test_samples = 25000

# pure_file_pattern = '/home/mreza/5G accelerator/ID_MEC/data generator/pure_data/pure_iq_samples_*.csv'
# mixed_file_pattern = '/home/mreza/5G accelerator/ID_MEC/data generator/mixed_data/mixed_iq_samples_*.csv'
# pure_file_new = '/home/mreza/5G accelerator/ID_MEC/data generator/New Data-Collection/rx_IQ_pure'
# mixed_file_new = '/home/mreza/5G accelerator/ID_MEC/data generator/New Data-Collection/rx_IQ_MIX'
# pure_file_old = '/home/mreza/5G accelerator/IQ_samples/data collected/5G_DL_IQ_no_jamming_0924.dat'
# mixed_file_old = '/home/mreza/5G accelerator/IQ_samples/data collected/5G_DL_IQ_with_periodic_jamming_0928_02.dat'


pure_file_new = r"C:\Users\Mohammadreza\Desktop\My Class\Proj-DC\My Works\My Papers\intrusion\data generator\New Data-Collection\rx_IQ_pure"
mixed_file_new = r"C:\Users\Mohammadreza\Desktop\My Class\Proj-DC\My Works\My Papers\intrusion\data generator\New Data-Collection\rx_IQ_MIX"


# Creating dataset instances for training and validation/testing
train_gen_instance  = ComplexNumbersDataset(filepath=pure_file_new, 
                                            sequence_length=sequence_length, 
                                            max_samples=max_train_samples, for_training=True, 
                                            process_method='data1')

combined_gen_instance = ComplexNumbersDataset(filepath=mixed_file_new, 
                                              sequence_length=sequence_length, 
                                              max_samples=max_samples, for_training=False, 
                                              process_method='data1')

# Creating DataLoader instances for batching
train_loader = DataLoader(train_gen_instance, batch_size=batch_size, shuffle=False, drop_last=True)
combined_loader = DataLoader(combined_gen_instance, batch_size=batch_size, shuffle=False, drop_last=True)


# Custom training loop in PyTorch
num_epochs = 2
for epoch in range(num_epochs):
    print(f"Using device: {device}")
    print(f"Epoch {epoch + 1}/{num_epochs}")
    model.train()  # Set the model to training mode
    
    total_loss = 0
    for batch_idx, (X_chunk, _) in enumerate(train_loader):  # Assuming autoencoder where input = target
        X_chunk = X_chunk.to(device)
        
        optimizer.zero_grad()
        reconstructed_x = model(X_chunk)
        #print('X_chunk.shape:', X_chunk.shape)
        #print('reconstructed_x.shape:', reconstructed_x.shape)
        loss = loss_function(reconstructed_x, X_chunk)  # Use X_chunk as both input and target
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        if batch_idx % 100 == 0:
            print(f"Batch {batch_idx}/{len(train_loader)}, Loss: {loss.item()}")

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}, Average Loss: {avg_loss}\n")






Using device: cpu
LSTMAutoencoder(
  (encoder_lstm1): LSTM(2, 50, batch_first=True)
  (self_attention): SelfAttentionLayer(
    (multihead_attention): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=50, out_features=50, bias=True)
    )
  )
  (encoder_lstm2): LSTM(50, 50, batch_first=True)
  (decoder_lstm1): LSTM(50, 50, batch_first=True)
  (decoder_lstm2): LSTM(50, 50, batch_first=True)
  (decoder_linear): Linear(in_features=50, out_features=2, bias=True)
)
Using device: cpu
Epoch 1/2
Batch 0/125, Loss: 1.0092170238494873
Batch 100/125, Loss: 0.22002004086971283
Epoch 1, Average Loss: 0.5579291649460792

Using device: cpu
Epoch 2/2
Batch 0/125, Loss: 0.8459047675132751
Batch 100/125, Loss: 0.201049342751503
Epoch 2, Average Loss: 0.3542669152021408



In [2]:

max_samples = 2500000  # Maximum samples to read (or None to read all)
max_test_samples = 2500000
combined_gen_instance = ComplexNumbersDataset(filepath=mixed_file_new, 
                                              sequence_length=sequence_length, 
                                              max_samples=max_samples, for_training=False, 
                                              process_method='data1')

# Creating DataLoader instances for batching
combined_loader = DataLoader(combined_gen_instance, batch_size=batch_size, shuffle=False, drop_last=True)


#num_predictions = 100  # or any other large number
#print(f"Number of predictions to be performed: {num_predictions}")
reconstruction_errors = []
all_X_chunk_test = []
all_X_chunk_pred = []
all_intrusion_flags = []

accumulated_real_chunk_errors = []
accumulated_X_chunk_test = []

threshold = None
#---------------------------------------------------------------------------------------------------
from jupyter_dash import JupyterDash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import torch
from torch.utils.data import DataLoader
from itertools import islice

# Your existing code for dataset setup and model definition...

# Initialize the Jupyter Dash app
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H1("Real-Time RNN Autoencoder Data and Prediction Visualization"),
    dcc.Graph(id='live-data-feed'),        
    dcc.Graph(id='live-comparison'),
    #dcc.Graph(id='accumulated-live-data-feed'),
    dcc.Graph(id='live-intrusion-detection'),
    dcc.Graph(id='live-prediction'),
    dcc.Interval(
        id='interval-component',
        interval=1*1000,
        n_intervals=0
    )
])

current_prediction_count = 0
num_predictions = 500  # Set this to the desired number of predictions
data_iterator = iter(combined_loader)  # Create an iterator for the DataLoader

def get_new_data():
    global data_iterator, current_prediction_count, num_predictions, combined_loader, model, device
    model.eval()  # Ensure the model is in evaluation mode
    if current_prediction_count >= num_predictions:
        print("Reached the maximum number of predictions, stopping updates.")
        return None, None, None  # Will cause the callback to prevent an update
    
    try:
        # Fetch the next batch of data
        X_chunk_test = next(data_iterator).to(device)
        X_chunk_pred = model(X_chunk_test).detach().cpu()
        chunk_errors = torch.mean((X_chunk_test - X_chunk_pred) ** 2, dim=1).cpu().numpy()
        X_chunk_test = X_chunk_test.cpu().numpy()
        X_chunk_pred = X_chunk_pred.cpu().numpy()
        current_prediction_count += 1  # Increment the count after a successful prediction
    except StopIteration:
        # If we've reached the end of the dataset before reaching num_predictions, reset the iterator
        data_iterator = iter(combined_loader)  # Reset the iterator
        current_prediction_count = 0  # Reset the prediction count
        return get_new_data()  # Retry fetching data recursively

    return X_chunk_test, X_chunk_pred, chunk_errors

# Callback for the data feed graph to plot only the real part
@app.callback(Output('live-data-feed', 'figure'),
              Input('interval-component', 'n_intervals'))
def update_data_feed(n):
    X_chunk_test, _, _ = get_new_data()
    print('X_chunk_test.shape:', X_chunk_test.shape)
    if X_chunk_test is None:
        raise dash.exceptions.PreventUpdate
    
    # Flatten the array and select only the real part (every second value starting from index 0)
    real_X_chunk_test = X_chunk_test[:,:,0].flatten()
    img_X_chunk_test = X_chunk_test[:,:,0].flatten()
    print('real_X_chunk_test.shape:', real_X_chunk_test.shape)
    print('img_X_chunk_test.shape:', img_X_chunk_test.shape)
    
    fig = go.Figure()
#     fig.add_trace(go.Scatter(x=list(range(len(X_chunk_test.flatten()))), y=X_chunk_test.flatten(), mode='lines', name='Input Data'))

    fig.add_trace(go.Scatter(
        x=list(range(len(real_X_chunk_test))),
        y=real_X_chunk_test,
        mode='lines',
        name='Input Data',
        line=dict(color='blue')
    ))

    fig.update_layout(
        title='Live Data Feed',
        title_font=dict(size=28),
        xaxis_title='Sample Index',
        xaxis=dict(
            title='Sample Index',
            title_font=dict(size=24),
            tickfont=dict(size=20),
        ),
        yaxis_title='Magnitude',
        yaxis=dict(
            title='Magnitude',
            title_font=dict(size=24),
            tickfont=dict(size=20),
        )
    )
    return fig



# Callback for the prediction graph to plot only the real part
@app.callback(Output('live-prediction', 'figure'),
              Input('interval-component', 'n_intervals'))
def update_prediction(n):
    _, X_chunk_pred, _ = get_new_data()
    if X_chunk_pred is None:
        raise dash.exceptions.PreventUpdate

    # Flatten the array and select only the real part (every second value starting from index 0)
    real_X_chunk_pred = X_chunk_pred[:,:,0].flatten()
    img_X_chunk_pred = X_chunk_pred[:,:,0].flatten()
    print('real_X_chunk_pred.shape:', real_X_chunk_pred.shape)
    print('img_X_chunk_pred.shape:', img_X_chunk_pred.shape)

    fig = go.Figure()
#     fig.add_trace(go.Scatter(x=list(range(len(X_chunk_pred.flatten()))), y=X_chunk_pred.flatten(), mode='lines', name='Predicted Data'))

    fig.add_trace(go.Scatter(
        x=list(range(len(real_X_chunk_pred))),
        y=real_X_chunk_pred,
        mode='lines',
        name='Predicted Data - Real Part',
        #line=dict(dash='dash', color='green')
        line=dict(color='green') 
    ))
    fig.update_layout(
        title='Live Prediction',
        title_font=dict(size=28),
        xaxis_title='Sample Index',
        xaxis=dict(
            title='Sample Index',
            title_font=dict(size=24),
            tickfont=dict(size=20),
        ),
        yaxis_title='Magnitude',
        yaxis=dict(
            title='Magnitude',
            title_font=dict(size=24),
            tickfont=dict(size=20), 
        )
    )
    return fig


#--------------------------------------------------------------------------------------
# @app.callback(Output('live-intrusion-detection', 'figure'),
#               Input('interval-component', 'n_intervals'))
# def update_intrusion_detection(n):
#     global accumulated_real_chunk_errors, threshold
#     _, _, chunk_errors = get_new_data()

#     if chunk_errors is None or not chunk_errors.any():
#         raise dash.exceptions.PreventUpdate
    
#     real_chunk_errors = chunk_errors[:, 0]
#     accumulated_real_chunk_errors.extend(real_chunk_errors)
    
#     if threshold is None:
#         # Assuming the first batch is representative for setting the threshold
#         threshold = np.percentile(accumulated_real_chunk_errors, 90)
#         print("Threshold set to:", threshold)

#     sequence_indices = list(range(len(accumulated_real_chunk_errors)))
    
#     fig_intrusion_detection = go.Figure()
#     fig_intrusion_detection.add_trace(go.Scatter(
#         x=sequence_indices,
#         y=accumulated_real_chunk_errors,
#         mode='lines+markers',
#         name='Real Reconstruction Error per Sequence',
#         line=dict(color='gray')
#     ))

#     # Add the constant threshold line
#     fig_intrusion_detection.add_hline(y=threshold, line_dash="dot", line_color="red", line_width=3,
#                                       #annotation_text=f"Threshold: {threshold:.2f}",
#                                       annotation_text=f"Threshold",
#                                       annotation_position="top right",
#                                       annotation_font=dict(size=14))

#     fig_intrusion_detection.update_layout(
#         title='Live Intrusion Detection',
#         title_font=dict(size=28),
#         xaxis_title='Sequence Index',
#         xaxis=dict(
#             title='Sequence Index',
#             title_font=dict(size=24),
#             tickfont=dict(size=20),
#         ),
#         yaxis_title='Reconstruction Error',
#         yaxis_type='linear',  # or 'log'
#         yaxis=dict(
#             title='Real Reconstruction Error',
#             title_font=dict(size=24),
#             tickfont=dict(size=20),
#         )
#     )
#     return fig_intrusion_detection
#-------------------------------------------------------------------------------------------------------

#-----------------------------------------------------------------------------------------------------------
@app.callback(Output('live-intrusion-detection', 'figure'),
              Input('interval-component', 'n_intervals'))
def update_intrusion_detection(n):
    global accumulated_real_chunk_errors, threshold
    _, _, chunk_errors = get_new_data()

    if chunk_errors is None or not chunk_errors.any():
        raise dash.exceptions.PreventUpdate

    real_chunk_errors = chunk_errors[:, 0]

    # If the first real batch is coming in and threshold not yet set, calculate it immediately
    if threshold is None and len(accumulated_real_chunk_errors) == 0:
        # Use the first batch of real errors to set the threshold
        threshold = np.percentile(real_chunk_errors, 75)
        print("Threshold set to:", threshold)

    # Continue to fill with zeros until 400 batches have been processed
    if len(accumulated_real_chunk_errors) < 400:
        # Extend with zeros instead of real errors
        accumulated_real_chunk_errors.extend([0] * len(real_chunk_errors))
    else:
        # After 400 batches, start accumulating actual errors
        accumulated_real_chunk_errors.extend(real_chunk_errors)

    sequence_indices = list(range(len(accumulated_real_chunk_errors)))

    fig_intrusion_detection = go.Figure()
    fig_intrusion_detection.add_trace(go.Scatter(
        x=sequence_indices,
        y=accumulated_real_chunk_errors,
        mode='lines+markers',
        name='Real Reconstruction Error per Sequence',
        line=dict(color='gray')
    ))

    # Add a constant threshold line if defined
    if threshold is not None:
        fig_intrusion_detection.add_hline(y=threshold, line_dash="dot", line_color="red", line_width=3,
                                          annotation_text="Threshold",
                                          annotation_position="top right",
                                          annotation_font_size=14)

    fig_intrusion_detection.update_layout(
        title='Live Intrusion Detection',
        title_font_size=28,
        xaxis_title='Sequence Index',
        xaxis=dict(
            title='Sequence Index',
            title_font=dict(size=24),
            tickfont=dict(size=20),
        ),
        yaxis_title='Reconstruction Error',
        yaxis=dict(
            title='Reconstruction Error',
            title_font=dict(size=24),
            tickfont=dict(size=20),
        ),
        yaxis_type='linear'  # This should be outside of the yaxis dict and directly in the update_layout call.
    )

    return fig_intrusion_detection


#-----------------------------------------------------------------------------------------------------------------

# Callback for the per-batch comparison graph
@app.callback(Output('live-comparison', 'figure'),
              [Input('interval-component', 'n_intervals')])
def update_comparison_graph(n):
    X_chunk_test, X_chunk_pred, _ = get_new_data()
    if X_chunk_test is None or X_chunk_pred is None:
        raise dash.exceptions.PreventUpdate
    
    # Extract both real parts for comparison
    real_X_chunk_test = X_chunk_test[:, :, 0].flatten()
    real_X_chunk_pred = X_chunk_pred[:, :, 0].flatten()

    fig = go.Figure()

    # Plot the real part of the test data
    fig.add_trace(go.Scatter(
        x=list(range(len(real_X_chunk_test))),
        y=real_X_chunk_test,
        mode='lines',
        name='Test Data - Real Part',
        line=dict(color='blue')
    ))

    # Plot the real part of the prediction data
    fig.add_trace(go.Scatter(
        x=list(range(len(real_X_chunk_pred))),
        y=real_X_chunk_pred,
        mode='lines',
        name='Prediction Data - Real Part',
        line=dict(color='orange')
    ))

    fig.update_layout(
        title='Live Data Prediction Per Batch',
        title_font=dict(size=28),
        xaxis_title='Sample Index',
        xaxis=dict(
            title='Sample Index',
            title_font=dict(size=24),
            tickfont=dict(size=20),
        ),
        yaxis_title='Magnitude',
        yaxis=dict(
            title='Magnitude',
            title_font=dict(size=24),
            tickfont=dict(size=20),
        ),
        legend=dict(
            font=dict(size=20)
        )
    )

    return fig
    
# Callback for the accumulated live data feed graph
@app.callback(Output('accumulated-live-data-feed', 'figure'),
              [Input('interval-component', 'n_intervals')])
def update_accumulated_data_feed(n):
    global accumulated_X_chunk_test
    X_chunk_test, _, _ = get_new_data()
    if X_chunk_test is None:
        raise dash.exceptions.PreventUpdate

    # Flatten the test data and accumulate it
    real_X_chunk_test = X_chunk_test[:,:,0].flatten()  # Assuming you only want the real part
    accumulated_X_chunk_test.extend(real_X_chunk_test)

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=list(range(len(accumulated_X_chunk_test))),
        y=accumulated_X_chunk_test,
        mode='lines',
        name='Accumulated Input Data',
        line=dict(color='purple')
    ))
    fig.update_layout(
        title='Accumulated Live Data Feed',
        title_font=dict(size=28),
        xaxis_title='Sample Index',
        xaxis=dict(
            title='Sample Index',
            title_font=dict(size=24),
            tickfont=dict(size=20),
        ),
        yaxis_title='Magnitude',
        yaxis=dict(
            title='Magnitude',
            title_font=dict(size=24),
            tickfont=dict(size=20),
        )
    )
    return fig

app.run_server(mode='external', debug=False)

#-----------------------------------------------------------------------------------------------------------

C:\Users\Mohammadreza\AppData\Local\Programs\Python\Python310\lib\site-packages\dash\dash.py:550: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



Threshold set to: 0.6979012340307236
real_X_chunk_pred.shape: (200,)
img_X_chunk_pred.shape: (200,)
X_chunk_test.shape: (20, 10, 2)
real_X_chunk_test.shape: (200,)
img_X_chunk_test.shape: (200,)
X_chunk_test.shape: (20, 10, 2)
real_X_chunk_test.shape: (200,)
img_X_chunk_test.shape: (200,)
real_X_chunk_pred.shape: (200,)
img_X_chunk_pred.shape: (200,)
X_chunk_test.shape: (20, 10, 2)
real_X_chunk_test.shape: (200,)
img_X_chunk_test.shape: (200,)
Dash app running on http://127.0.0.1:8050/
real_X_chunk_pred.shape: (200,)
img_X_chunk_pred.shape: (200,)
X_chunk_test.shape: (20, 10, 2)
real_X_chunk_test.shape: (200,)
img_X_chunk_test.shape: (200,)
real_X_chunk_pred.shape: (200,)
img_X_chunk_pred.shape: (200,)
X_chunk_test.shape: (20, 10, 2)
real_X_chunk_test.shape: (200,)
img_X_chunk_test.shape: (200,)
real_X_chunk_pred.shape: (200,)
img_X_chunk_pred.shape: (200,)
X_chunk_test.shape: (20, 10, 2)
real_X_chunk_test.shape: (200,)
img_X_chunk_test.shape: (200,)
real_X_chunk_pred.shape: (200,)
img

X_chunk_test.shape: (20, 10, 2)
real_X_chunk_test.shape: (200,)
img_X_chunk_test.shape: (200,)
real_X_chunk_pred.shape: (200,)
img_X_chunk_pred.shape: (200,)
real_X_chunk_pred.shape: (200,)
img_X_chunk_pred.shape: (200,)
X_chunk_test.shape: (20, 10, 2)
real_X_chunk_test.shape: (200,)
img_X_chunk_test.shape: (200,)
real_X_chunk_pred.shape: (200,)
img_X_chunk_pred.shape: (200,)
X_chunk_test.shape: (20, 10, 2)
real_X_chunk_test.shape: (200,)
img_X_chunk_test.shape: (200,)
X_chunk_test.shape: (20, 10, 2)
real_X_chunk_test.shape: (200,)
img_X_chunk_test.shape: (200,)
real_X_chunk_pred.shape: (200,)
img_X_chunk_pred.shape: (200,)
X_chunk_test.shape: (20, 10, 2)
real_X_chunk_test.shape: (200,)
img_X_chunk_test.shape: (200,)
real_X_chunk_pred.shape: (200,)
img_X_chunk_pred.shape: (200,)
X_chunk_test.shape:real_X_chunk_pred.shape: (200,)
img_X_chunk_pred.shape: (200,)
 (20, 10, 2)
real_X_chunk_test.shape: (200,)
img_X_chunk_test.shape: (200,)
X_chunk_test.shape: (20, 10, 2)
real_X_chunk_test.sh

X_chunk_test.shape: (20, 10, 2)
real_X_chunk_test.shape: (200,)
img_X_chunk_test.shape: (200,)
real_X_chunk_pred.shape: (200,)
img_X_chunk_pred.shape: (200,)
X_chunk_test.shape: (20, 10, 2)
real_X_chunk_test.shape: (200,)
img_X_chunk_test.shape: (200,)
real_X_chunk_pred.shape: (200,)
img_X_chunk_pred.shape: (200,)
X_chunk_test.shape: (20, 10, 2)
real_X_chunk_test.shape: (200,)
img_X_chunk_test.shape: (200,)
real_X_chunk_pred.shape: (200,)
img_X_chunk_pred.shape: (200,)
X_chunk_test.shape: (20, 10, 2)
real_X_chunk_test.shape: (200,)
img_X_chunk_test.shape: (200,)
real_X_chunk_pred.shape: (200,)
img_X_chunk_pred.shape: (200,)
real_X_chunk_pred.shape: (200,)
img_X_chunk_pred.shape: (200,)
X_chunk_test.shape: (20, 10, 2)
real_X_chunk_test.shape: (200,)
img_X_chunk_test.shape: (200,)
X_chunk_test.shape: (20, 10, 2)
real_X_chunk_test.shape: (200,)
img_X_chunk_test.shape: (200,)
real_X_chunk_pred.shape: (200,)
img_X_chunk_pred.shape: (200,)
real_X_chunk_pred.shape: (200,)
img_X_chunk_pred.sha

Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum numb

...

Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum numb

Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum numb

Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum numb

Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum numb

Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum number of predictions, stopping updates.
Reached the maximum numb